In [2]:
2+2/3

2.6666666666666665

In [11]:
import numpy as np
import pandas as pd
from typing import List, Dict
from dataclasses import dataclass

In [4]:
test_timetable = pd.read_csv("test_graph.csv")

In [17]:
class Stop:
    def __init__(self, name: str, latitude: float, longitude: float) -> None:
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
    def __str__(self) -> str:
        return self.name + "[" + str(self.latitude) + ", " + str(self.longitude) + "]"
    def __repr__(self) -> str:
        return self.name + ";" + str(self.latitude) + ";" + str(self.longitude)
    def __hash__(self) -> int:
        return hash(str(self.name))
    def __eq__(self, __value: object) -> bool:
        return isinstance(__value, Stop) and str(self) == str(__value)
    
@dataclass
class Route:
    line: str
    destination: Stop
    departure_time: str
    arrival_time: str

In [18]:
all_stops: Dict[Stop, Route] = {}
for stop in test_timetable.itertuples():
    start_stop: Stop = Stop(stop.start_stop, stop.start_stop_lat, stop.start_stop_lon) 
    end_stop: Stop = Stop(stop.end_stop, stop.end_stop_lat, stop.end_stop_lon)
    if start_stop not in all_stops.keys():
        all_stops[start_stop] = [end_stop] 
    else:
        all_stops[start_stop].append(end_stop)

In [19]:
all_stops

{Chełmońskiego;51.10388356;17.09071195: [Tramwajowa;51.10446678;17.08466997],
 Tramwajowa;51.10446678;17.08466997: [ZOO;51.10556081;17.0778982],
 ZOO;51.10556081;17.0778982: [Hala Stulecia;51.10708825;17.07346452],
 Hala Stulecia;51.10708825;17.07346452: [Kliniki - Politechnika Wrocławska;51.10971261;17.06511141],
 Kliniki - Politechnika Wrocławska;51.10971261;17.06511141: [PL. GRUNWALDZKI;51.11144435;17.05995961,
  PL. GRUNWALDZKI;51.1111569;17.06146483],
 PL. GRUNWALDZKI;51.11144435;17.05995961: [most Grunwaldzki;51.11021216;17.05538488,
  Piastowska;51.11536582;17.06087806],
 Piastowska;51.11536582;17.06087806: [Prusa;51.11947637;17.05740455],
 Prusa;51.11947637;17.05740455: [Wyszyńskiego;51.12191852;17.05294165],
 Wyszyńskiego;51.12191852;17.05294165: [Nowowiejska;51.12423914;17.0448911],
 Nowowiejska;51.12423914;17.0448911: [Słowiańska;51.12354187;17.04021724],
 Słowiańska;51.12354187;17.04021724: [DWORZEC NADODRZE;51.12431442;17.03503321],
 DWORZEC NADODRZE;51.12431442;17.0350332

In [51]:
stops

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

In [23]:
test_timetable.head

<bound method NDFrame.head of     Unnamed: 0       company  line departure_time arrival_time  \
0       107472  MPK Tramwaje     4       19:52:00     19:53:00   
1       107473  MPK Tramwaje     4       19:53:00     19:55:00   
2       107474  MPK Tramwaje     4       19:55:00     19:56:00   
3       107475  MPK Tramwaje     4       19:56:00     19:58:00   
4       107476  MPK Tramwaje     4       19:58:00     20:00:00   
5       107477  MPK Tramwaje     4       20:00:00     20:01:00   
6        56906  MPK Tramwaje     1       20:26:00     20:28:00   
7        56907  MPK Tramwaje     1       20:28:00     20:30:00   
8        56908  MPK Tramwaje     1       20:30:00     20:32:00   
9        56909  MPK Tramwaje     1       20:32:00     20:33:00   
10       56910  MPK Tramwaje     1       20:33:00     20:35:00   
11       56911  MPK Tramwaje     1       20:35:00     20:36:00   
12       56912  MPK Tramwaje     1       20:36:00     20:38:00   
13       56913  MPK Tramwaje     1       20:38

NameError: name 'Stop' is not defined

In [27]:
test_arr

array([[107472, 'MPK Tramwaje', 4, '19:52:00', '19:53:00',
        'Chełmońskiego', 'Tramwajowa', 51.10388356, 17.09071195,
        51.10446678, 17.08466997],
       [107473, 'MPK Tramwaje', 4, '19:53:00', '19:55:00', 'Tramwajowa',
        'ZOO', 51.10446678, 17.08466997, 51.10556081, 17.0778982],
       [107474, 'MPK Tramwaje', 4, '19:55:00', '19:56:00', 'ZOO',
        'Hala Stulecia', 51.10556081, 17.0778982, 51.10708825,
        17.07346452],
       [107475, 'MPK Tramwaje', 4, '19:56:00', '19:58:00',
        'Hala Stulecia', 'Kliniki - Politechnika Wrocławska',
        51.10708825, 17.07346452, 51.10971261, 17.06511141],
       [107476, 'MPK Tramwaje', 4, '19:58:00', '20:00:00',
        'Kliniki - Politechnika Wrocławska', 'PL. GRUNWALDZKI',
        51.10971261, 17.06511141, 51.11144435, 17.05995961],
       [107477, 'MPK Tramwaje', 4, '20:00:00', '20:01:00',
        'PL. GRUNWALDZKI', 'most Grunwaldzki', 51.11144435, 17.05995961,
        51.11021216, 17.05538488],
       [56906, 'M

In [26]:
for row in test_arr:
    stop = Stop(row[latitude])

NameError: name 'Stop' is not defined

In [5]:
real_timetable = pd.read_csv("connection_graph (1).csv")

C:\Users\48791\AppData\Local\Temp\ipykernel_11972\3244852323.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  real_timetable = pd.read_csv("connection_graph (1).csv")


In [6]:
real_timetable.head

<bound method NDFrame.head of         Unnamed: 0               company line departure_time arrival_time  \
0                0          MPK Autobusy    A       20:52:00     20:53:00   
1                1          MPK Autobusy    A       20:53:00     20:54:00   
2                2          MPK Autobusy    A       20:54:00     20:55:00   
3                3          MPK Autobusy    A       20:55:00     20:57:00   
4                4          MPK Autobusy    A       20:57:00     20:59:00   
...            ...                   ...  ...            ...          ...   
996515      996515  DLA Kąty Wrocławskie  967       18:38:00     18:39:00   
996516      996516  DLA Kąty Wrocławskie  967       18:39:00     18:41:00   
996517      996517  DLA Kąty Wrocławskie  967       18:41:00     18:42:00   
996518      996518  DLA Kąty Wrocławskie  967       18:42:00     18:43:00   
996519      996519  DLA Kąty Wrocławskie  967       18:43:00     18:45:00   

                         start_stop          

In [15]:
print(real_timetable['arrival_time'].agg(['min', 'max']))

min    03:36:00
max    30:05:00
Name: arrival_time, dtype: object


In [ ]:
print("Values range for departure time: " + real_timetable['departure_time'].min() + " - " + real_timetable['departure_time'].max())
print("Values range for arrival time: " + real_timetable['arrival_time'].min() + " - " + real_timetable['arrival_time'].max())
print("Values range for start-stop latitude: " + str(real_timetable['start_stop_lat'].min()) + " - " + str(real_timetable['start_stop_lat'].max()))
print("Values range for start-stop longitude: " + str(real_timetable['start_stop_lon'].min()) + " - " + str(real_timetable['start_stop_lon'].max()))

Przedzial departure time: 03:34:00 - 30:03:00
Przedzial arrival time: 03:36:00 - 30:05:00
Przedzial start-stop latitude: 50.98042601 - 51.26556584
Przedzial start-stop longitude: 16.695853 - 17.2855453
